# Python中階-爬取更多資料


## 中正大學資管系 (20181021) 大綱

咦，內容不只有一頁啊？我要怎麼爬更多內容的資料呢？

## 上一頁、下一頁的按鈕分析

連入 ptt web 有很多頁面，但以剛剛例子只進入了最新的那個頁面，如果今天要寫一個聰明的爬蟲或許要做一個更彈性化的功能，讓爬蟲可以進入更多頁面取的更多的訊息。首先起手式還是要分析網頁結構。

![](images/CHROME3.png)

可以發現上一頁這個按鈕後面就很清楚的顯示`上一頁`網址的名稱，這不是太棒了嗎？(有人會問為什麼只有上一頁？下一頁內容是空白，那是因為版面的首頁就是最新的一頁呀)

In [2]:
import requests
from requests_html import HTML

url = 'https://www.ptt.cc/bbs/movie/index.html'
resp = requests.get(url)

In [4]:
html = HTML(html=resp.text)
pages = html.find('.action-bar a.btn.wide')
pages #這個元素的標籤共有四個值，我們要取 index= 1 的才是上一頁，所以我們要把裡面的 href 取出來

[<Element 'a' class=('btn', 'wide') href='/bbs/movie/index1.html'>,
 <Element 'a' class=('btn', 'wide') href='/bbs/movie/index7200.html'>,
 <Element 'a' class=('btn', 'wide', 'disabled')>,
 <Element 'a' class=('btn', 'wide') href='/bbs/movie/index.html'>]

In [6]:
link = pages[1].attrs.get('href')
link

'/bbs/movie/index7200.html'

## 如何把網址做結合

大家會發現我們取出的網址只有相對路徑，其實這在網站中很常見，意思是在這個超連結都是這個頁面相對於這個網站位置，類似
```
root--
      |
      a---
          |
          a1.html
      b---
          |
          b1.html
          
root/a/a1.html
root/b/b1.html
```
為什麼，其實這很好解釋維護的問題。如果每個超連結都是給完整位置，改天網站 root 改名了那所有的頁面就要改掉了，所以都會用相對路徑來表示
因此必須要我們手動還原這個網址，才能透過 `get` 方式讀到，所以我們使用到 `split()` 大絕招，把網址從 `/bbs` 直接切一半，然後把剛剛我們取出的連結位置做結合

In [9]:
next_page = resp.url.split('/bbs')[0] + link
next_page

'https://www.ptt.cc/bbs/movie/index7200.html'

但這個方法也有缺陷，因為哪天邏輯不是這樣時大概就會出現 `bug`，因此另一個比較乾淨做法是針對 next_page 使用一個新邏輯，這會用到 urlib 這個套件

In [19]:
import urllib
domain = 'https://www.ptt.cc/'
next_page = urllib.parse.urljoin(domain, link)
next_page

'https://www.ptt.cc/bbs/movie/index7200.html'

## 建立一個函式來爬所有資料

如果成功完成這件事，那把它獨立成一個 function，未來應該就能夠重複利用，但原本程式是設計爬取單一個頁面，所以我們還要修改程式讓主程式能夠確定要爬多少頁

+ 建立函式來取的下一頁
+ 爬蟲主程式根據參數，循序爬取所有頁面

In [22]:
def parser_next_page(resp):
    html = HTML(html=resp.text)
    pages = html.find('.action-bar a.btn.wide')
    link = pages[1].attrs.get('href')
    next_page = urllib.parse.urljoin(domain, link)
    return next_page

parser_next_page(resp)

'https://www.ptt.cc/bbs/movie/index7200.html'

In [27]:
widths = [
        (126,    1), (159,    0), (687,     1), (710,   0), (711,   1),
        (727,    0), (733,    1), (879,     0), (1154,  1), (1161,  0),
        (4347,   1), (4447,   2), (7467,    1), (7521,  0), (8369,  1),
        (8426,   0), (9000,   1), (9002,    2), (11021, 1), (12350, 2),
        (12351,  1), (12438,  2), (12442,   0), (19893, 2), (19967, 1),
        (55203,  2), (63743,  1), (64106,   2), (65039, 1), (65059, 0),
        (65131,  2), (65279,  1), (65376,   2), (65500, 1), (65510, 2),
        (120831, 1), (262141, 2), (1114109, 1),
]


def calc_len(string):
    def chr_width(o):
        global widths
        if o == 0xe or o == 0xf:
            return 0
        for num, wid in widths:
            if o <= num:
                return wid
        return 1
    return sum(chr_width(ord(c)) for c in string)


def pretty_print(push, title, date, author):
    pattern = '%3s\t%s%s%s\t%s'
    padding = ' ' * (50 - calc_len(title))
    print(pattern % (push, title, padding, date, author))


def fetch(url):
    response = requests.get(url)
    return response


def parser_article_meta(entry):
    context = {
        'title': entry.find('div.title', first=True).text,
        'push': entry.find('div.nrec', first=True).text,
        'date': entry.find('div.date', first=True).text,
        'author': entry.find('div.author', first=True).text,
    }
    return context


def get_page_meta(resp):
    html = HTML(html=resp.text)
    post_entries = html.find('div.r-ent')

    for entry in post_entries:
        meta = parser_article_meta(entry)
        #print(meta)
        pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])
        
        
def main(url, nums):
    for _ in range(nums):
        print(f"第{_+1}頁, {url}")
        resp = fetch(url)
        if resp.status_code == 200:
            get_page_meta(resp)
            url = parser_next_page(resp)
        else:
            print(resp.status_code)
        
        
if __name__ == '__main__':
    url = 'https://www.ptt.cc/bbs/movie/index.html'
    main(url=url, nums=5)

第1頁, https://www.ptt.cc/bbs/movie/index.html
  2	[公告] 水桶公告 20181017                          10/17	VOT1077
  4	[新聞] 登陸月球50年《電影哆啦A夢 大雄的月球探     10/17	hoanbeh
 17	Re: [贈票] 【極智對決】 台北贈票                  10/17	rapsd520
   	新聞文章請以新發文方式-V <Reewalker>              10/17	-
  4	[請益] 李小龍傳                                   10/17	hsinofkids
  8	[討論] 最經典的系列作有哪些                       10/17	assggy
 75	[新聞] 重建熱蘭遮城將投入135億 魏德聖：2024       10/17	purue
  2	[贈票] 極智對決 週六台北贈票                      10/17	WAV
  1	[情報] 2018 亞太銀幕獎 入圍名單                   10/17	qpr322
  4	[問片] 孕婦車禍流產，找人復仇的血腥片             10/17	shuffling
 23	[新聞] 華倫夫婦加入《安娜貝爾3》                  10/17	shengchiu303
   	[資訊]2018新北市影視人才培育課程結訓儀式          10/17	unalaba
  6	[片單] 類似上海灘賭聖一片兩拍的電影?              10/17	Marchosias
 10	[討論] 王家衛真喜歡用劇組人員的名字               10/17	joey0602
  2	[討論] 和家人看電影時看到激情的場面？             10/17	pully37
  1	[版規] 電影版版規 201808                          8/28	VOT1077
   	[公告] 電影版板規修訂說明                         8/28	VOT1077
 33	[公告] 關於特定影片負(好)雷

有更好的寫法嗎？